# 《数据采集与清洗》
## 第一次作业内容： 网页抓取
### 具体目标：
+ 用``urllib.requests``库发起一次``get``请求，输出``response``文本信息；
+ 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息；
+ 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容；
+ 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果；
+ 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果；
+ (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。

### 注：
+ 代码要有注释，结果要有分析；
+ 本次作业提交截至时间：2020年3月10日(星期二)；
+ 文件命名规则: 班级号+学号+姓名+作业序号，示例：``1_20188989899_张三_1``；
+ 提交方式：1班发送至邮箱：632994085@qq.com；2班发送至邮箱：786888939@qq.com.

### 问题1：用``urllib.requests``库发起一次``get``请求，输出``response``文本信息。

In [8]:
import requests
response = requests.get("http://httpbin.org/get")
print(response.text)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.22.0", 
    "X-Amzn-Trace-Id": "Root=1-5e6513c2-243185be0f747f6d57e39788"
  }, 
  "origin": "211.97.123.227", 
  "url": "http://httpbin.org/get"
}



响应返回了响应头headers等信息、域名及url

这里对结果进行分析。

### 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息。

In [10]:
data = {
    "name" : "Jasson",
    "age" : "20"
}
r = requests.post("http://httpbin.org/post",data=data)
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "age": "20", 
    "name": "Jasson"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "18", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.22.0", 
    "X-Amzn-Trace-Id": "Root=1-5e651ba2-99358b76fe44e3bcceec7233"
  }, 
  "json": null, 
  "origin": "211.97.123.227", 
  "url": "http://httpbin.org/post"
}



成功返回了POST请求信息，其中form部分就是提交的数据data，表面POST请求成功发送了

这里对结果进行分析。

### 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容。 

In [19]:
import requests
s = requests.Session()
s.get('http://httpbin.org/cookies/set/number/123456789')#设置一个cookie,名称叫作 number,内容是 123456789
response = s.get('http://httpbin.org/cookies')  #再次请求了该网址，获取当前的 Cookies。
print(response.text)

{
  "cookies": {
    "number": "123456789"
  }
}



成功获取到设置的Cookies

这里对结果进行分析。

### 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果。


In [37]:
import requests,re
count = 5  #控制站点层数
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')  #把正则表达式编译成正则表达对象，方便复用
seed = 'https://www.baidu.com'
stack = [seed]  #用栈数据结构来深度遍历
while len(stack)>0 and count>0:
    try:
        url = stack.pop(-1)  #取栈顶元素(该层网页最后一个url)
        html = requests.get(url).text #获取get请求得到的html原文本
        new_urls = r.findall(html) #找该站点所有符合设定正则形式的url
        stack.extend(new_urls) #压栈
        number = len(new_urls)
        count = count - 1
        print(url+ "  该网站下的子网站个数"+ "%d"  %number)
    except Exception as e:
        print(url)
        print(e)

https://www.baidu.com  该网站下的子网站个数12
http://jianyi.baidu.com/  该网站下的子网站个数8
http://rj.baidu.com/soft/detail/11843.html?ald  该网站下的子网站个数13
http://www.baidu.com/link?url=01vNBVXR2eaJxETl9PI3hcrvKCcwaJIKk5FkpO7l5YI_Q_pC24ogIBoZxI0LAu5oYpAdhRH42nzxAqfui0YnHK  该网站下的子网站个数218
http://index.baidu.com  该网站下的子网站个数3


这里对结果进行分析。

### 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果。 

In [52]:
#去重算法
import requests,re
count = 5 #控制遍历层数
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
seed = 'http://httpbin.org/'
queue = [seed]
used = set() #用集合特点模拟hash表
storage = {}
while len(queue)>0 and count>0:
    try:
        new_url_ =[]  #用来计数当前层去重后的url个数
        url = queue.pop(0)  #取队列中首次访问的url
        used.add(url)
        html = requests.get(url).text
        new_urls = r.findall(html)
        for new_url in new_urls: #遍历该层所有站点，将新发现的且不相同的url加入队列中
            if new_url not in used and new_url not in queue:
                queue.append(new_url)
                new_url_.append(new_url)
        count = count - 1
        print(url,len(new_url_))  #输出与该站点“平行”（同层）且互异的网站个数
    except Exception as e:
        print(url)
        print(e)

http://httpbin.org/ 4
https://fonts.googleapis.com/css?family=Open+Sans:400,700|Source+Code+Pro:300,600|Titillium+Web:400,600,700 0
https://github.com/requests/httpbin 55
https://kennethreitz.org 45
https://github.com/rochacbruno/flasgger 30


In [41]:
#不去重
import requests,re
count = 5 #控制遍历层数
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
seed = 'http://httpbin.org/'
queue = [seed]
used = set()
while len(queue)>0 and count>0:
    try:
        url = queue.pop(0)  #取队列中的起始url （遍历各层网页时首次访问的url）
        used.add(url)
        html = requests.get(url).text
        new_urls = r.findall(html)
        queue.extend(new_urls) #将处于与当前访问站点相同层数的站点加入到队列中（不论是否相同）
        print(url,len(new_urls))  #输出与当前访问站点“平行”（同层）（有可能重复）的网站个数
        count = count - 1
    except Exception as e:
        print(url)
        print(e)

http://httpbin.org/ 4
https://fonts.googleapis.com/css?family=Open+Sans:400,700|Source+Code+Pro:300,600|Titillium+Web:400,600,700 0
https://github.com/requests/httpbin 59
https://kennethreitz.org 51
https://github.com/rochacbruno/flasgger 71


这里对结果进行分析。

### (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。 

In [ ]:
class Seaching():
    def DFS_Url(url):
        '''
        输入一个种子网站，进行去重深度遍历
        '''
        count = 5  #控制站点层数
        r = re.compile(r'href=[\'"]?(http[^\'" >]+)')  #把正则表达式编译成正则表达对象，方便复用
        seed = url
        stack = [seed]  #用栈数据结构来深度遍历
        used = set()
        while len(stack)>0 and count>0:
            try:
                new_url_ =[] #计数当前层去重后的url个数
                url = stack.pop(-1)  #取栈顶元素(该层网页最后一个url)
                used.add(url)
                html = requests.get(url).text #获取get请求得到的html原文本
                new_urls = r.findall(html) #找该站点所有符合设定正则形式的url
                for new_url in new_urls:
                    if new_url not in used and new_url not in stack:
                        stack.append(new_url)
                number = len(new_urls)
                count = count - 1
                print(url+ "  该网站下的子网站个数"+ "%d"  %number)
            except Exception as e:
                print(url)
                print(e)

这里对结果进行分析。